In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer, explained_variance_score, max_error
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from scipy.stats import boxcox, yeojohnson, skew, pearsonr, spearmanr, kendalltau
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import LayerNormalization, Dropout
import shap
import lime
import lime.lime_tabular
from keras_tuner.tuners import RandomSearch
from keras_tuner import HyperParameters
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import FunctionTransformer
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder

# 1. Enhanced Data Generation
def generate_data(num_samples=3000, temporal=False):

    data = {
        'Engine_Speed_RPM': np.random.randint(1000, 3000, num_samples),
        'Load_Percentage': np.random.uniform(0, 100, num_samples),
        'Injection_Timing_Deg': np.random.uniform(5, 20, num_samples),
        'Compression_Ratio': np.random.uniform(14, 20, num_samples),
        'Fuel_Type': np.random.choice(['Diesel', 'BioDiesel_20', 'BioDiesel_40'], num_samples, p=[0.5, 0.3, 0.2]),
        'Ambient_Temperature_C': np.random.uniform(20, 40, num_samples),
        'Air_Fuel_Ratio': np.random.uniform(14, 18, num_samples),
        'Intake_Air_Pressure_kPa': np.random.uniform(90, 110, num_samples),
        'Coolant_Temperature_C': np.random.uniform(70, 90, num_samples)
    }
    df = pd.DataFrame(data)

    # Create temporal effects
    if temporal:
        time = np.arange(num_samples) / num_samples * 10  # Time in arbitrary unit
        df['Engine_Speed_RPM'] += (np.sin(time) * 50).astype(int)
        df['Ambient_Temperature_C'] += (np.cos(time) * 5).astype(float)

    # Simulate efficiency with interactions and non-linearity
    df['BioThermal_Efficiency'] = 40 + 0.02 * df['Engine_Speed_RPM'] + 0.15 * df['Load_Percentage'] - 0.8*df['Injection_Timing_Deg'] + 0.3 * df['Compression_Ratio'] - 0.05 * df['Ambient_Temperature_C'] + 0.08 * df['Air_Fuel_Ratio'] + 0.005 * df['Engine_Speed_RPM']*df['Load_Percentage']  + np.random.normal(0, 2, num_samples)  # Interaction + noise

    # Make efficiency dependent on fuel type with offsets
    df['BioThermal_Efficiency'] = df.apply(lambda row: row['BioThermal_Efficiency'] + 2 if row['Fuel_Type'] == 'BioDiesel_20' else row['BioThermal_Efficiency'], axis=1)
    df['BioThermal_Efficiency'] = df.apply(lambda row: row['BioThermal_Efficiency'] + 4 if row['Fuel_Type'] == 'BioDiesel_40' else row['BioThermal_Efficiency'], axis=1)

    # Add missing values
    for col in df.columns[:-1]:
        mask = np.random.choice([True, False], num_samples, p=[0.1, 0.9])  # 10% missing values
        df.loc[mask, col] = np.nan

    #Introduce outliers
    for col in df.select_dtypes(include='number').columns:
        if col != "BioThermal_Efficiency":
            df_col = df[col]
            outliers_indicies = np.random.choice(df_col.index, int(0.01 * num_samples),replace = False)
            df.loc[outliers_indicies,col] = df.loc[outliers_indicies,col] + (df.loc[outliers_indicies,col] * 1.5)
    return df

df = generate_data(temporal = True)

# 2. Enhanced Data Preprocessing
def preprocess_data(df):
    #Missing values imputation
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    numeric_cols.remove('BioThermal_Efficiency')

    imputer_knn = KNNImputer(n_neighbors=5)
    df[numeric_cols] = imputer_knn.fit_transform(df[numeric_cols])

    # Scaling and Transformation
    X = df.drop('BioThermal_Efficiency', axis=1)
    y = df['BioThermal_Efficiency']

    #One hot encoding

    categorical_cols = X.select_dtypes(include='object').columns
    one_hot_encoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

    encoder_transformer = ColumnTransformer(
        transformers = [
            ('onehot', one_hot_encoder, categorical_cols)
        ],
        remainder = 'passthrough'
    )
    X_encoded = encoder_transformer.fit_transform(X)
    column_names = encoder_transformer.get_feature_names_out()
    X_encoded = pd.DataFrame(X_encoded, columns = column_names)

    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    #Robust scaling with QuantileTransformer
    quantile_transformer = QuantileTransformer(output_distribution = 'normal', n_quantiles = 500)
    X_train_scaled_quantile = quantile_transformer.fit_transform(X_train)
    X_test_scaled_quantile = quantile_transformer.transform(X_test)

    #Box-Cox or Yeo-Johnson
    power_transform = PowerTransformer()
    X_train_scaled_power = power_transform.fit_transform(X_train)
    X_test_scaled_power = power_transform.transform(X_test)

    return X_train_scaled, X_test_scaled, X_train_scaled_quantile, X_test_scaled_quantile, X_train_scaled_power, X_test_scaled_power, y_train, y_test, scaler, encoder_transformer

X_train, X_test, X_train_quantile, X_test_quantile, X_train_power, X_test_power, y_train, y_test, scaler, encoder = preprocess_data(df)


# 3. Enhanced EDA
def eda(df):
    print("\n--- Data Summary ---")
    print(df.describe(include='all'))

    #Interactive scatter plot
    fig = px.scatter_matrix(df.select_dtypes(include='number'),title="Interactive Scatter Matrix")
    fig.show()

    print("\n--- Correlation Analysis ---")
    plt.figure(figsize=(12, 8))
    sns.heatmap(df.corr(numeric_only = True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title("Correlation Matrix")
    plt.show()

    #Skewness check
    skew_values = df.select_dtypes(include='number').apply(lambda x: skew(x, nan_policy='omit'))
    print("\n--- Skewness of Numerical Features ---")
    print(skew_values)


    print("\n---Distribution of Numerical Features---")
    for col in df.select_dtypes(include='number').columns:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f'Distribution of {col}')
        plt.show()

    # Boxplot of Fuel Type vs. Efficiency
    sns.boxplot(x = df['Fuel_Type'], y = df['BioThermal_Efficiency'])
    plt.show()

    # Calculate Pearson, Spearman, Kendall correlations with target variable
    target_col = "BioThermal_Efficiency"
    print("\n--- Correlation Coefficients with Target ---")
    for col in df.select_dtypes(include='number').columns:
      if col != target_col:
        pearson, _ = pearsonr(df[col].dropna(), df[target_col].dropna())
        spearman, _ = spearmanr(df[col].dropna(), df[target_col].dropna())
        kendall, _ = kendalltau(df[col].dropna(), df[target_col].dropna())

        print(f"{col}: Pearson = {pearson:.3f}, Spearman = {spearman:.3f}, Kendall = {kendall:.3f}")

    # Time series analysis
    if 'time' in df.columns:
       plt.figure(figsize=(10,6))
       plt.plot(df['time'],df['BioThermal_Efficiency'])
       plt.title('Time Series plot of Efficiency')
       plt.xlabel('Time')
       plt.ylabel("BioThermal_Efficiency")
       plt.show()


eda(df)

# 4. Enhanced Machine Learning Models
def train_ml_models(X_train, y_train, X_test, y_test):
    models = {
        "Linear Regression": LinearRegression(),
        "Ridge Regression": Ridge(),
        "Lasso Regression": Lasso(),
        "Elastic Net Regression": ElasticNet(),
        "Polynomial Regression":  PolynomialFeatures(degree=2),
        "SVR": SVR(),
        "Random Forest": RandomForestRegressor(random_state=42),
        "Gradient Boosting": GradientBoostingRegressor(random_state=42),
        "AdaBoost": AdaBoostRegressor(random_state=42),
        "Bagging Regressor": BaggingRegressor(random_state=42)
    }

    tuned_models = {}
    for name, model in models.items():

        print(f"\n------{name} Training-------")
        if name == "Polynomial Regression":
            poly = model
            X_train_poly = poly.fit_transform(X_train)
            X_test_poly = poly.transform(X_test)
            lr_model = LinearRegression()
            lr_model.fit(X_train_poly,y_train)
            y_pred = lr_model.predict(X_test_poly)
            tuned_models[name] = lr_model,poly

        elif name == "SVR":
            param_grid = {'kernel':['rbf', 'linear'],
                           'C':[0.1, 1, 10],
                           'gamma':['scale', 'auto', 0.1,1]
                           }
            grid = GridSearchCV(model, param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_jobs = -1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)
            tuned_models[name] = best_model

        elif name == "Random Forest":
            param_grid = {'n_estimators':[100, 200, 500],
                          'max_depth':[None, 5, 10],
                          'min_samples_split':[2, 5, 10],
                          'min_samples_leaf':[1, 2, 4],
                          'max_features':['sqrt', 'log2', None]
                          }
            grid = RandomizedSearchCV(model, param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_iter = 10, n_jobs = -1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)
            tuned_models[name] = best_model

        elif name == "Gradient Boosting":
            param_grid = {'n_estimators':[100, 200, 500],
                          'learning_rate':[0.01, 0.05, 0.1],
                          'max_depth':[3,5,8],
                          'min_samples_split':[2, 5, 10],
                          'min_samples_leaf':[1, 2, 4],
                          'max_features':['sqrt', 'log2', None]
                          }
            grid = RandomizedSearchCV(model, param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_iter = 10, n_jobs = -1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)
            tuned_models[name] = best_model

        elif name in ["Ridge Regression", "Lasso Regression", "Elastic Net Regression"]:
            param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0], 'fit_intercept': [True, False]}
            if name == "Elastic Net Regression":
                param_grid["l1_ratio"] = [0,0.25,0.5,0.75,1]
            grid = GridSearchCV(model, param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_jobs = -1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)
            tuned_models[name] = best_model

        elif name in ["AdaBoost", "Bagging Regressor"]:
            param_grid = {'n_estimators': [50, 100, 200],
                         'learning_rate': [0.01, 0.1, 1.0]
                        } if name == "AdaBoost" else {'n_estimators':[10, 50, 100]}

            grid = RandomizedSearchCV(model, param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_iter = 10 if name == 'AdaBoost' else 3, n_jobs = -1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)
            tuned_models[name] = best_model

        else :
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            tuned_models[name] = model
        evaluate_model(name,y_test,y_pred)
    return tuned_models

def evaluate_model(name, y_test, y_pred):
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        evs = explained_variance_score(y_test, y_pred)
        max_err = max_error(y_test, y_pred)

        print(f"------{name} Evaluation-------")
        print(f"MSE: {mse:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MAE: {mae:.2f}")
        print(f"R-squared: {r2:.2f}")
        print(f"Explained Variance Score: {evs:.2f}")
        print(f"Max Error: {max_err:.2f}")

tuned_ml_models = train_ml_models(X_train, y_train, X_test, y_test)

# 5. Enhanced Deep Learning Model
def build_dl_model(input_shape,hp):
        model = models.Sequential()

        model.add(layers.Input(shape=(input_shape,)))

        for i in range(hp.Int('num_layers', 2, 6)):
            model.add(layers.Dense(units = hp.Int(f'units_{i}',32,256,step = 32),
                                  activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'selu'])))
            model.add(LayerNormalization())
            model.add(Dropout(rate = hp.Float(f'dropout_{i}', min_value = 0.0, max_value= 0.5, step=0.1)))

        model.add(layers.Dense(1)) #Regression outout

        optimizer_choice = hp.Choice('optimizer', ['adam', 'adamw'])
        learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')

        if optimizer_choice == 'adam':
            optimizer = optimizers.Adam(learning_rate=learning_rate)
        elif optimizer_choice == 'adamw':
            optimizer = optimizers.AdamW(learning_rate=learning_rate)

        model.compile(optimizer=optimizer, loss='mse', metrics=[MeanSquaredError(), MeanAbsoluteError()])
        return model

def train_dl_model(X_train, y_train, X_test, y_test):
    input_shape = X_train.shape[1]

    def tuner_function(hp):
        return build_dl_model(input_shape, hp)

    tuner = RandomSearch(
        tuner_function,
        objective='val_mean_squared_error',
        max_trials=10,
        executions_per_trial=1,
        directory = 'model_dir',
        project_name = 'bio_thermal_tuner'
    )

    tuner.search(X_train,y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[callbacks.EarlyStopping(monitor='val_loss',patience = 10, restore_best_weights = True)]
        )

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    best_model = tuner.hypermodel.build(best_hps)
    history = best_model.fit(X_train,y_train,
                            validation_split=0.2,
                            epochs = 100,
                            batch_size=32,
                            verbose = 0,
                             callbacks=[callbacks.EarlyStopping(monitor='val_loss',patience = 10, restore_best_weights = True)]
                            )
    y_pred = best_model.predict(X_test).flatten()
    evaluate_model("Deep Learning MLP",y_test,y_pred)
    return best_model, history

dl_model, dl_history = train_dl_model(X_train, y_train, X_test, y_test)

# 6. Generative Model
def build_generator(latent_dim, n_features):
    model = models.Sequential([
        layers.Dense(128, input_dim=latent_dim),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum = 0.8),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum = 0.8),
        layers.Dense(n_features, activation='tanh')
    ])
    return model

def build_discriminator(n_features):
    model = models.Sequential([
        layers.Dense(256, input_dim = n_features),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def train_gan(df, epochs=5000, batch_size=32, latent_dim=10):

    X = df.drop('BioThermal_Efficiency', axis=1)

    #One hot encoding
    categorical_cols = X.select_dtypes(include='object').columns
    one_hot_encoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

    encoder_transformer = ColumnTransformer(
        transformers = [
            ('onehot', one_hot_encoder, categorical_cols)
        ],
        remainder = 'passthrough'
    )
    X_encoded = encoder_transformer.fit_transform(X)
    column_names = encoder_transformer.get_feature_names_out()
    X_encoded = pd.DataFrame(X_encoded, columns = column_names)
    X_encoded = X_encoded.values.astype('float32')

    n_features = X_encoded.shape[1]
    generator = build_generator(latent_dim, n_features)
    discriminator = build_discriminator(n_features)

    discriminator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

    discriminator.compile(loss='binary_crossentropy', optimizer = discriminator_optimizer)
    discriminator.trainable = False #Discriminator training will happen seperately

    gan_input = layers.Input(shape = (latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = models.Model(gan_input,gan_output)
    gan.compile(loss='binary_crossentropy', optimizer = generator_optimizer)

    for epoch in range(epochs):
        idx = np.random.randint(0,X_encoded.shape[0],batch_size)
        real_images = X_encoded[idx]

        # Generate fake data
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)

        # Train discriminator
        discriminator.trainable = True
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)

        # Train generator
        discriminator.trainable = False
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, D Loss: {d_loss:.3f}, G Loss: {g_loss:.3f}")
    return generator,encoder_transformer

generator,encoder_transformer = train_gan(df)

def generate_synthetic_samples(generator, latent_dim=10, num_samples=1000, encoder_transformer = None):
        noise = np.random.normal(0, 1, (num_samples, latent_dim))
        generated_data = generator.predict(noise)

        if encoder_transformer:
            generated_data = pd.DataFrame(generated_data, columns=encoder_transformer.get_feature_names_out())

            original_cols = [col.split('__')[0] for col in encoder_transformer.get_feature_names_out()]
            unique_cols = list(set(original_cols))

            generated_df = pd.DataFrame(index = generated_data.index)

            for col in unique_cols:
                if col in original_cols:
                    if col in df.columns:
                       if  df[col].dtype == 'object':
                            sub_df = generated_data[[c for c in generated_data.columns if c.startswith(col)]]
                            generated_df[col] = sub_df.idxmax(axis=1).apply(lambda x : x.split('__')[1])
                       else:
                            generated_df[col] = generated_data[[c for c in generated_data.columns if c.startswith(col)]].sum(axis=1)

            return generated_df
        else:
          return generated_data
generated_samples = generate_synthetic_samples(generator,num_samples = 1000, encoder_transformer = encoder_transformer)


# 7. Model Interpretability
def interpret_model(model, X_train, X_test, y_train,y_test, explainer_type='shap'):
    print("\n--- Model Interpretability ---")

    if explainer_type == "shap":
        explainer = shap.TreeExplainer(model) if isinstance(model, RandomForestRegressor) or isinstance(model, GradientBoostingRegressor) else shap.Explainer(model, X_train)
        shap_values = explainer.shap_values(X_test)
        shap.summary_plot(shap_values, X_test, plot_type="bar")

    elif explainer_type == "lime":
         explainer = lime.lime_tabular.LimeTabularExplainer(
             X_train, feature_names = X_train.columns, class_names = ['efficiency'], mode = 'regression'
         )
         index = np.random.choice(X_test.index)
         exp = explainer.explain_instance(X_test.loc[index], model.predict, num_features = 10)
         exp.show_in_notebook(show_table = True)

    # if explainer_type == "dl":
    #     explainer = shap.DeepExplainer(model, X_train)
    #     shap_values = explainer.shap_values(X_test)
    #     shap.summary_plot(shap_values, X_test, plot_type="bar")

for name, model in tuned_ml_models.items():
    if name == "Polynomial Regression":
        model, poly = model
        X_train_poly = poly.transform(X_train)
        X_test_poly = poly.transform(X_test)
        interpret_model(model, X_train_poly, X_test_poly, y_train, y_test)
    else:
        interpret_model(model, X_train, X_test, y_train, y_test)

#Interpret DL model
#interpret_model(dl_model, X_train, X_test, y_train, y_test, explainer_type = "dl")

# 8. Discussion on Generative AI and Model Combination
def generative_ai_discussion(generated_samples):
    print("\n--- Generative AI Discussion ---")
    print("GANs have been used to generate synthetic data with similar statistical properties as our training data.")
    print("Here's how this generated data can be used:")
    print(" 1. Data Augmentation: The generated data can be combined with the real dataset to expand the dataset's scope, potentially leading to more robust regression model.")
    print(" 2. Exploration of New Parameter Spaces: The GAN can help explore areas of engine parameters that we haven't seen in our data.")
    print(" 3. Model Combination: We can also use the generated data to train a separate regression model and then average or ensemble that model with our models trained on original data.")
    print("The GAN is not for direct prediction but to augment or aid in the exploration of parameter space")

generative_ai_discussion(generated_samples)

Trial 9 Complete [00h 00m 33s]
val_mean_squared_error: 20003.474609375

Best val_mean_squared_error So Far: 17638.048828125
Total elapsed time: 00h 05m 58s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
3                 |2                 |num_layers
160               |160               |units_0
relu              |selu              |activation_0
0.1               |0                 |dropout_0
224               |128               |units_1
relu              |relu              |activation_1
0                 |0.3               |dropout_1
adamw             |adamw             |optimizer
2.9065e-05        |0.00080078        |learning_rate
96                |224               |units_2
relu              |selu              |activation_2
0.3               |0.4               |dropout_2
64                |256               |units_3
relu              |selu              |activation_3
0                 |0.2               |dropout_3
160               |256            

KeyboardInterrupt: 

In [2]:
!pip install lime keras-tuner scikit-learn plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.1 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=8e240cac80b7de5795ae5839c9d786091ea5c6cba5eaf45495da402e639e6fc0
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
